In [ ]:
!pip install dropbox

In [14]:
import numpy as np
import pandas as pd
import itertools as it
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, learning_curve
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, precision_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pylab as plt

import dropbox

import plotly.graph_objects as go

standarscaler = StandardScaler()

from warnings import filterwarnings
filterwarnings('ignore')

In [15]:
Data = pd.read_csv('https://raw.githubusercontent.com/mcuscagua/Aprendizaje_Automatico/master/Trabajo_2/Data.csv')
Data = Data.set_index('Date')
Y = Data['Class']
X = Data.drop('Class', axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

X_train_Stand = standarscaler.fit_transform(X_train)
X_test_Stand = standarscaler.transform(X_test)

In [24]:
Neurons = [x+1 for x in range(10)]
all_combinations = []
for i in range(2):
    all_combinations += [p for p in it.product(Neurons, repeat=i+1)]
    
TSZ_LC_Vector = np.ndarray((5,len(all_combinations)))
TrSCr_LC_Vector = np.ndarray((5,len(all_combinations)))
TeSCr_LC_Vector = np.ndarray((5,len(all_combinations)))
AUC_vector = np.repeat(0,len(all_combinations))
Accuracy_vector = np.repeat(0,len(all_combinations))

In [25]:
for i in range(len(all_combinations)):
    MLPC = MLPClassifier(hidden_layer_sizes=all_combinations[i],
                             learning_rate = 'constant',
                             learning_rate_init = 0.2,
                             max_iter = 50,
                             tol = 1e-2,
                             random_state=0)

    MLPP = MLPClassifier(hidden_layer_sizes=all_combinations[i],
                             learning_rate = 'constant',
                             learning_rate_init = 0.2,
                             max_iter = 50,
                             tol = 1e-2,
                             random_state=0)

    MLPC.fit(X_train_Stand, Y_train)
    y_score = MLPC.predict_proba(X_test_Stand)
    y_pred = MLPC.predict(X_test_Stand)

    train_sizes, train_scores, test_scores = learning_curve(MLPP, X, Y)
    
    TSZ_LC_Vector[:,i] = train_sizes
    TrSCr_LC_Vector[:,i] = np.median(train_scores, axis=1)
    TeSCr_LC_Vector[:,i] = np.median(test_scores, axis=1)
    AUC_vector[i] = roc_auc_score(Y_test, y_pred)
    Accuracy_vector[i] = accuracy_score(Y_test, y_pred)

In [29]:
AUC_vector[i] = roc_auc_score(Y_test, y_pred)

array([[ 330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,  330.,
         330.,  330.],
       [1073., 1073., 1073., 1073., 1073., 1073., 1073., 1073., 1073.,
        1073., 1073., 1073., 1073., 1073., 1073., 1073

In [23]:
MLPC = MLPClassifier(hidden_layer_sizes=(10),
                         learning_rate = 'constant',
                         learning_rate_init = 0.2,
                         max_iter = 50,
                         tol = 1e-2,
                         random_state=0)

MLPC.fit(X_train_Stand, Y_train)
y_score = MLPC.predict_proba(X_test_Stand)
y_pred = MLPC.predict(X_test_Stand)

AUC = roc_auc_score(Y_test, y_pred)
Acc = accuracy_score(Y_test, y_pred)
print(AUC)

0.5052910052910053


In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.savefig('The one.png')
    return plt

In [ ]:
ja = plot_learning_curve(MLPP, 'LEARNING CURVE', X, Y)

In [ ]:
ja.savefig('The one.png')